In [1]:
import torch

import mowl
mowl.init_jvm('10g')
from mowl.datasets import PathDataset, Dataset

from src.utils import *
from src.elmodule import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

cpu


In [3]:
# dataset_name = 'family'
# dataset_name = 'pizza'
# dataset_name = 'OWL2DL-1'
dataset_name = 'lubm_filtered'

In [5]:
experiments = get_experimets(dataset_name)

**Models**

**Eval**

**Experiments: Box2EL**

In [5]:
for experiment in experiments: 
    dataset_name = experiment['dataset_name']
    file_name = experiment['file_name']
    format_ = experiment['format_']
    add_noise = experiment['add_noise']                                                                                                               
   
    dataset = PathDataset(ontology_path=f'datasets/bin/{file_name}_train.owl', # we add noise to train
                          testing_path=f'datasets/bin/{dataset_name}_test.owl',
                          validation_path=f'datasets/bin/{dataset_name}_val.owl')
    
    train_ont = preprocess_ontology_el(dataset.ontology)
    test_ont = preprocess_ontology_el(dataset.testing)
    valid_ont = preprocess_ontology_el(dataset.validation)

    dataset = Dataset(train_ont, testing=test_ont, validation=valid_ont)
    
    box2el = ElModel(dataset, 
                     module_name='box2el', 
                     dim=200, 
                     margin=0.1, 
                     batch_size=4096*8, 
                     test_batch_size=32, 
                     epochs=500, 
                     learning_rate=0.001,
                     device=device)
    
    box2el._train()
    
    print(f'{file_name}:')
    print('Membership:')
    metrics_membership = box2el._eval('membership')
    print('Subsumption:')
    metrics_subsumption = box2el._eval('subsumption')
    print('Link Prediction:')
    metrics_link_prediction = box2el._eval('link_prediction')
    print()
    save_results(metrics_subsumption, metrics_membership, metrics_link_prediction, f'models/results/box2el/{file_name}.txt')

In [6]:
#######################################################

In [ ]:
dataset = PathDataset(ontology_path=f'datasets/bin/{dataset_name}_train.owl', 
                      testing_path=f'datasets/bin/{dataset_name}_test.owl',
                      validation_path=f'datasets/bin/{dataset_name}_val.owl')

train_ont = preprocess_ontology_el(dataset.ontology)
test_ont = preprocess_ontology_el(dataset.testing)
valid_ont = preprocess_ontology_el(dataset.validation)

dataset = Dataset(train_ont, testing=test_ont, validation=valid_ont)

box2el = ElModel(dataset, 
                 module_name='box2el', 
                 dim=256, 
                 margin=0.1, 
                 batch_size=4096*8, 
                 test_batch_size=32, 
                 epochs=500, 
                 learning_rate=0.0001,
                 device=device)

In [ ]:
metrics_link_prediction = box2el._eval('membership')

In [ ]:
metrics_link_prediction = box2el._eval('subsumption')

In [ ]:
metrics_link_prediction = box2el._eval('link_prediction')

In [ ]:
box2el._train()

In [ ]:
metrics_link_prediction = box2el._eval('membership')

In [ ]:
metrics_link_prediction = box2el._eval('link_prediction')

In [ ]:
metrics_link_prediction = box2el._eval('subsumption')

In [14]:
#######################################################

In [6]:
dataset = PathDataset(ontology_path=f'datasets/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0_Publication0.nt', 
                      testing_path=f'datasets/jena_inference_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0_Publication0.nt')


train_ont = preprocess_ontology_el(dataset.ontology)
test_ont = preprocess_ontology_el(dataset.testing)

dataset = Dataset(train_ont, testing=test_ont)

box2el = ElModel(dataset, 
                 module_name='box2el', 
                 dim=256, 
                 margin=0.1, 
                 batch_size=4096*8, 
                 test_batch_size=32, 
                 epochs=500, 
                 learning_rate=0.0001,
                 device=device)

In [7]:
metrics_link_prediction = box2el._eval('membership')

MRR: 0.595, Hits@1: 0.286, Hits@5: 1.000, Hits@10: 1.000


In [8]:
metrics_link_prediction = box2el._eval('link_prediction')

ZeroDivisionError: division by zero

In [21]:
dataset = PathDataset(ontology_path=f'MyJenaProject/input_toy_example/U0WC0D2AP3_without_tbox.ttl', 
                      testing_path=f'MyJenaProject/output_toy_example/U0WC0D2AP3_filtered_inferred.ttl')


train_ont = preprocess_ontology_el(dataset.ontology)
test_ont = preprocess_ontology_el(dataset.testing)

dataset = Dataset(train_ont, testing=test_ont)

box2el = ElModel(dataset, 
                 module_name='box2el', 
                 dim=256, 
                 margin=0.1, 
                 batch_size=4096*8, 
                 test_batch_size=32, 
                 epochs=500, 
                 learning_rate=0.0001,
                 device=device)

In [22]:
metrics_link_prediction = box2el._eval('membership') # Observation 2: membership performs better if TBox is not included in training set

MRR: 0.560, Hits@1: 0.310, Hits@5: 1.000, Hits@10: 1.000


In [10]:
metrics_link_prediction = box2el._eval('subsumption')

INFO:root:Reverse translation. Ignoring axiom: RangeAxiom*(161 94)
INFO:root:'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyRa' object has no attribute 'getSubClass'
INFO:root:Reverse translation. Ignoring axiom: RangeAxiom*(162 95)
INFO:root:'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyRa' object has no attribute 'getSubClass'
INFO:root:Reverse translation. Ignoring axiom: RangeAxiom*(137 96)
INFO:root:'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyRa' object has no attribute 'getSubClass'
INFO:root:Reverse translation. Ignoring axiom: RangeAxiom*(172 128)
INFO:root:'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyRa' object has no attribute 'getSubClass'
INFO:root:Reverse translation. Ignoring axiom: FunctionalObjectProperty*(53)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '53'.
INFO:root:Reverse translation. Ignoring axiom: RangeAxiom*(163 96)
INFO:root:'uk.ac.manchester.cs.owl.owlapi.O

IndexError: too many indices for tensor of dimension 1

In [56]:
abox_ds_train = box2el.get_abox_data("train")
abox_dl_train = FastTensorDataLoader(*abox_ds_train, batch_size=box2el.batch_size, shuffle=True)

In [57]:
main_dl = abox_dl_train

In [58]:
for batch_data in main_dl:
    ind_idxs, labels, property_labels = batch_data
    break

In [59]:
module = BoxSquaredELModule(box2el.nb_classes, box2el.nb_roles, embed_dim = box2el.dim)

In [60]:
abox_logits = box2el.module.abox_forward(ind_idxs.to(box2el.device), 'membership')
abox_loss = F.binary_cross_entropy_with_logits(abox_logits, labels.to(box2el.device))

In [61]:
abox_logits_lp = box2el.module.abox_forward(ind_idxs.to(box2el.device), 'link_prediction')
abox_loss_lp = F.binary_cross_entropy_with_logits(abox_logits_lp, property_labels.to(box2el.device))